In [1]:
import ee
import folium
import geopandas as gpd
import tensorflow as tf

2023-10-09 14:32:04.332877: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 14:32:07.623813: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-10-09 14:32:07.624884: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
ee.Initialize()

# 1. Convert the sample image to a TensorFlow record

In [202]:
# Read in the image from EE
sample_image = ee.Image("projects/esg-satelite/assets/mars/modelling/model_input/test1")

In [203]:
# Create "stack"
image_stack = ee.Image.cat(
    [
        sample_image
    ]
)

In [204]:
image_stack = image_stack.float()

In [205]:
task = ee.batch.Export.image.toCloudStorage(
    image=image_stack,
    description="TensorFlow record of sample area of THEMIS Tempe Terra.",
    bucket="esg-satelite-data-warehouse",
    fileNamePrefix=f"mars/predictions/features/themis_sample_1/themis_sample_1",
    fileFormat = "TFRecord",
    region = ee.Feature(sample_image.geometry()).geometry(),
    scale = 500,
    formatOptions = {
        "patchDimensions": [256,256],
        "maxFileSize": 104857600,
        "compressed": True,
    }
)

In [206]:
task.start()

In [210]:
task.status()

{'state': 'COMPLETED',
 'description': 'TensorFlow record of sample area of THEMIS Tempe Terra.',
 'creation_timestamp_ms': 1696864526199,
 'update_timestamp_ms': 1696864549163,
 'start_timestamp_ms': 1696864536149,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://console.developers.google.com/storage/browser/esg-satelite-data-warehouse/mars/predictions/features/themis_sample_1/'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 0.10350694507360458,
 'id': 'EWZK7CTRCPJ7LQCBAJWS6XBS',
 'name': 'projects/earthengine-legacy/operations/EWZK7CTRCPJ7LQCBAJWS6XBS'}

## 1.1. Plotting on Folium

In [51]:
# Creating add_ee_layer function
# This allows us to visualise EE objects on a Folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee_image_object.getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [18]:
Map = folium.Map() 

Map.add_ee_layer(
    sample_image,
    {},
    "Tempe Terra"
)

Map.add_ee_layer(
    ee.FeatureCollection(sample_image.geometry()),
    {"color": "red"},
    "Geometry"
)

_ = folium.LayerControl().add_to(Map)


In [19]:
Map

# 2. Read in image

##### Reference: https://github.com/google/earthengine-community/blob/master/guides/linked/TF_demo1_keras.ipynb "Read the JSON mixer file"

In [231]:
import json
import numpy as np

In [232]:
json_file = "gs://esg-satelite-data-warehouse/mars/predictions/features/themis_sample_1/themis_sample_1.json"

In [233]:
json_text = !gsutil cat {json_file}

In [234]:
mixer = json.loads(json_text.nlstr)

In [235]:
# Get relevant info from the JSON mixer file.
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width * patch_height, 1]

In [236]:
patch_dimensions_flat

[65536, 1]

In [237]:
BANDS = ["b1"]


In [238]:
# Note that the tensors are in the shape of a patch, one patch for each band.
image_columns = [
  tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) 
    for k in BANDS
]

In [239]:
# Parsing dictionary.
image_features_dict = dict(zip(BANDS, image_columns))

In [240]:
# Note that you can make one dataset from many files by specifying a list.
image_dataset = tf.data.TFRecordDataset([json_file], compression_type= "GZIP")

In [241]:
# Parsing function.
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

# Parse the data into tensors, one long tensor per patch.
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

In [242]:
image_dataset

<ParallelMapDataset element_spec={'b1': TensorSpec(shape=(65536, 1), dtype=tf.float32, name=None)}>

In [243]:
# Break our long tensors into many little ones.
image_dataset = image_dataset.flat_map(
  lambda features: tf.data.Dataset.from_tensor_slices(features)
) 

In [244]:
image_dataset

<FlatMapDataset element_spec={'b1': TensorSpec(shape=(1,), dtype=tf.float32, name=None)}>

In [247]:
# Turn the dictionary in each record into a tuple without a label.
image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

# Turn each patch into a batch.
image_dataset = image_dataset.batch(patch_width * patch_height)

In [248]:
image_dataset

<BatchDataset element_spec=(TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None),)>

In [249]:
np.expand_dims(np.concatenate([x for x, _ in image_dataset], axis=0)[i], axis=0)

DataLossError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} inflate() failed with error -3: incorrect header check [Op:IteratorGetNext]

# 3. Get model

In [250]:
import tensorflow as tf

In [251]:
model = tf.keras.models.load_model("gs://esg-satelite-data-warehouse/mars/models/sample_model_1")

In [252]:
# Run prediction in batches, with as many steps as there are patches.
predictions = model.predict(image_dataset, steps=patches, verbose=1)

ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 2079, in predict_step
        return self(x, training=False)
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer 'max_pooling2d_5' (type MaxPooling2D).
    
    Negative dimension size caused by subtracting 2 from 1 for '{{node model_1/ma